In [151]:
#-- Modules & Packages

import pandas as pd 
import numpy as np 

from glob import glob 
from time import strftime, sleep 
from datetime import datetime
from pandas_datareader import data as pdr
from pandas.tseries.offsets import BDay 

import yfinance as yf
yf.pdr_override()

#-- Plotting 

import seaborn as sns
import dash 
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html 
import dash_bootstrap_components as dbc 
import plotly.express as px 
import dash_table 
import plotly.graph_objects as go 
from jupyter_dash import JupyterDash

In [152]:
# pip install jupyter_dash
# pip install dash_bootstrap_components

This Dashboard relies on synthetic transactions data, recurrent USD amount to purchase each asset at the start of each month since 2022; 
this approach is obviously not the most realistic, as we create a data set of imaginary buy-spot regardless of market conditions without
selLing, the aim is to provide a realistic and full-fledged dashboard to monitor the performance of the portfolio. 


I can build on top of this code by selecting from a transaction file, buy & sell dates,
buy & sell prices, fees, cumulative units, cumulative cost, average price of purchase, average selling price, etc...

<h5>Fake Transactions Dataset.<br>

<h6>Amount per crypto:<br>
BTC-USD 50000 $ <br>
ETH-USD 50000 $ <br>
AVAX-USD 10000 $ <br>
OP-USD 10000 $ <br>
ARB-USD 10000 $.

In [166]:
portfolio_tickers = ['BTC-USD', 'ETH-USD', 'OP-USD','ARB-USD','AVAX-USD']

one_year = datetime.today().year - 1
today = datetime.today()
start_x = datetime(2022, 1, 1)
end_x = today

start_assets = datetime(2022,1,1)
end_assets = today

start_ytd = datetime(one_year, 12, 31) + BDay(1)

# def get_txs_data(tickers, startDate, interval):
def data_x(ticker):
    #-- monthly interval '1mo'
    data = (yf.download(ticker, start = start_x, interval= '1mo'))
    data.columns = [x.lower() for x in data.columns]
    data.drop(columns = ['open','high','low','volume','adj close'], axis = 1, inplace = True)
    data['ticker'] =  ticker
    data['type'] = 'Buy'
    data['val_transact'] = 50000
    data['quantity'] = data['val_transact']/ data['close'] 
    data['prev_units'] = data['quantity'].shift(1)
    data['cml_units'] = data['quantity'].cumsum()
    data['prev_cost'] = data['val_transact'].shift(1)
    data['cml_cost'] = data['val_transact'].cumsum()
    #-- applying spot taker-fees in Binance Spot Markets
    data['cost_transact'] = data['val_transact'] * 0.0009500
    data['cml_invested'] = data['val_transact'].cumsum() - data['cost_transact']
    data['cost_unit'] = data['cml_cost'] / data['quantity']
    data['cum_position_val'] = data['cml_units'] * data['close']
    data['gain_loss'] = data['cum_position_val'] - data['cml_invested'] 
    data['yield'] = (data['gain_loss'] / data['cml_invested']) - 1
    #-- running, cumulative mean to accurately assess the avg price through each purchase date/row
    data['avg_price'] = data['close'].expanding().mean()
    data['current_value'] = data['close'] * data['cml_units']

    #-- Assigning recurrent exact amount for each crypto asset
    if ticker == 'BTC-USD':
      data['val_transact'] = 50000
    elif ticker == 'ETH-USD':
      data['val_transact'] = 50000
    elif ticker == 'AVAX-USD':
      data['val_transact'] = 10000
    elif ticker == 'LTC-USD':
      data['val_transact'] = 10000
    return data

#-- Fetching data for BTC-USD
btcusd = data_x('BTC-USD')
#-- Fetching data for ETH-USD
ethusd = data_x('ETH-USD')
#-- Fetching data for LTC-USD
ltcusd = data_x('LTC-USD')

#-- Fetching data for AVAX-USD
avaxusd = data_x('AVAX-USD')


first_concat =  pd.concat([btcusd, ethusd], axis =1 )
second_concat = pd.concat([ltcusd, avaxusd], axis = 1)
all_df = [first_concat,second_concat]
transactions_df = pd.concat(all_df)
#-- Saving transactions dataframe
transactions_df.to_excel(r"D:\_datasets\Projects_Coding\portfolio_dashboard_base2\inputs\dummy_transactions.xlsx")
#-- Previewing transactions dataframe
transactions_df.tail(5) 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,close,ticker,type,val_transact,quantity,prev_units,cml_units,prev_cost,cml_cost,cost_transact,...,prev_cost,cml_cost,cost_transact,cml_invested,cost_unit,cum_position_val,gain_loss,yield,avg_price,current_value
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-01,72.798439,LTC-USD,Buy,10000,686.827914,719.988232,15848.446665,50000.0,1200000,47.5,...,50000.0,1200000,47.5,1199952.5,925.077850,2.585573e+06,1.385620e+06,0.154729,27.388319,2.585573e+06
2024-01-01,66.753044,LTC-USD,Buy,10000,749.029511,686.827914,16597.476176,50000.0,1250000,47.5,...,50000.0,1250000,47.5,1249952.5,829.047489,2.274483e+06,1.024531e+06,-0.180344,27.619262,2.274483e+06
2024-02-01,79.940895,LTC-USD,Buy,10000,625.462099,749.029511,17222.938275,50000.0,1300000,47.5,...,50000.0,1300000,47.5,1299952.5,1064.320053,2.857645e+06,1.557693e+06,0.198269,28.131421,2.857645e+06
2024-03-01,105.183403,LTC-USD,Buy,10000,475.360167,625.462099,17698.298442,50000.0,1350000,47.5,...,50000.0,1350000,47.5,1349952.5,1460.981552,3.827377e+06,2.477425e+06,0.835194,29.093606,3.827377e+06
2024-04-01,76.960564,LTC-USD,Buy,10000,649.683391,475.360167,18347.981833,50000.0,1400000,47.5,...,50000.0,1400000,47.5,1399952.5,972.072556,2.505619e+06,1.105666e+06,-0.210212,29.294438,2.505619e+06


Data Collecting and Saving.

In [168]:
#-- Cleaning columns string names
def clean_headers(df):
    df.columns = df.columns.str.strip().str.lower().str.replace('.', '').str.replace('(','').str.replace(')','').str.replace(' ','_').str.replace('_/_','/')

#-- Getting timestamps for file names before saving 
def get_tmstmp():
    now = datetime.now().strftime('%Y-%m-%d_%Hh%Mm')
    return now

last_file = (r'D:\_datasets\Projects_Coding\portfolio_dashboard_base2\inputs/dummy_transactions.xlsx')
# print(last_file[-(len(last_file)) + (last_file.rfind('/')+1):])

all_ops = pd.read_excel(last_file)
all_ops.date = pd.to_datetime(all_ops.Date, format = "%d/%m/%Y")

all_tickers = list(all_ops['ticker'].unique())
blackList = ['PEPE-USD', 'DOGE-USD', 'ADA-USD']
final_tickers = [tick for tick in all_tickers if tick not in blackList]
print("Traded {} different cryptos".format(len(all_tickers)))
#-- All transactions without blacklisted assets
final_filtered = all_ops[~all_ops.ticker.isin(blackList)]

Traded 2 different cryptos


C:\Users\hp\AppData\Local\Temp\ipykernel_13024\358629261.py:14: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



In [169]:

portfolio_tickers = ['BTC-USD', 'ETH-USD','AVAX-USD', 'LTC-USD']

one_year = datetime.today().year - 1
today = datetime.today()
start_x = datetime(2023, 1, 1)
end_x = today

start_assets = datetime(2023,1,1)
end_assets = today

start_ytd = datetime(one_year, 12, 31) + BDay(1)

def get_data(tickers, startDate, endDate):
    def data(ticker):
        data = (pdr.get_data_yahoo(ticker, start = startDate, end = endDate))
        data.columns = [x.lower() for x in data.columns]
        data.drop(columns = ['adj close'], axis = 1, inplace = True)
        return data
    datum = map(data, tickers)
    return(pd.concat(datum, keys = tickers, names = ['ticker', 'date']))

all_prices_df = get_data(portfolio_tickers, start_x, end_x)
all_prices_df

# #-- Saving all asset prices separately
# for ticker in final_filtered:
#     all_prices_df.loc[ticker].to_csv("D:\_datasets\Projects_Coding\portfolio_dashboard_base2\outputs\{}_price_hist.csv".format(ticker))

# all_prices_df['ticker']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


open          high           low         close  \
ticker  date                                                                 
BTC-USD 2023-01-01  16547.914062  16630.439453  16521.234375  16625.080078   
        2023-01-02  16625.509766  16759.343750  16572.228516  16688.470703   
        2023-01-03  16688.847656  16760.447266  16622.371094  16679.857422   
        2023-01-04  16680.205078  16964.585938  16667.763672  16863.238281   
        2023-01-05  16863.472656  16884.021484  16790.283203  16836.736328   
...                          ...           ...           ...           ...   
LTC-USD 2024-04-10     97.498123     98.134644     94.457336     96.699760   
        2024-04-11     96.700424     99.800484     95.044495     98.689102   
        2024-04-12     98.688545     99.288963     80.812927     86.269569   
        2024-04-13     86.263977     86.492859     71.311768     77.606583   
        2024-04-14     77.597496     80.364578     74.138649     79.914330   

                         volume  
ticker  date                     
BTC-USD 2023-01-01   9244361700  
        2023-01-02  12097775227  
        2023-01-03  13903079207  
        2023-01-04  18421743322  
        2023-01-05  13692758566  
...                         ...  
LTC-USD 2024-04-10    571530949  
        2024-04-11    533522657  
        2024-04-12   1052316398  
        2024-04-13   1220477380  
        2024-04-14    833631968  

[1880 rows x 5 columns]

In [189]:
MEGA_DICT = {}  # you have to create it first
min_date = '2022-01-01'  # optional
# TX_COLUMNS = ['date','ticker', 'cashflow', 'cml_units', 'cml_cost', 'gain_loss']
TX_COLUMNS = ['Date','ticker', 'cml_units','val_transact', 'cml_cost', 'gain_loss']

tx_filt = all_ops[TX_COLUMNS]  # keeping just the most relevant ones for now

for ticker in portfolio_tickers:
    prices_df = all_prices_df[all_prices_df.index.get_level_values('ticker').isin([ticker])].reset_index()
    ## Can add more columns like volume!
    PX_COLS = ['date', 'close']
    prices_df = prices_df[PX_COLS].set_index(['date'])
    # Making sure we get sameday transactions
    tx_df = tx_filt[tx_filt.ticker==ticker].groupby('Date').agg({'val_transact': 'sum',
                                                                 'cml_units': 'last',
                                                                 'cml_cost': 'last',
                                                                 'gain_loss': 'sum'})
    # Merging price history and transactions dataframe
    tx_and_prices = pd.merge(prices_df, tx_df, how='outer', left_index=True, right_index=True).fillna(0)
    # This is to fill the days that were not in our transaction dataframe
    tx_and_prices['cml_units'] = tx_and_prices['cml_units'].replace(to_replace=0, method='ffill')
    tx_and_prices['cml_cost'] = tx_and_prices['cml_cost'].replace(to_replace=0, method='ffill')
    tx_and_prices['gain_loss'] = tx_and_prices['gain_loss'].replace(to_replace=0, method='ffill')
    # Cumulative sum for the val_transact
    tx_and_prices['val_transact'] = tx_and_prices['val_transact'].cumsum()
    tx_and_prices['avg_price'] = (tx_and_prices['cml_cost']/tx_and_prices['cml_units'])
    tx_and_prices['mktvalue'] = (tx_and_prices['cml_units']*tx_and_prices['close'])
    tx_and_prices = tx_and_prices.add_prefix(ticker+'_')
    # Once we're happy with the dataframe, add it to the dictionary
    MEGA_DICT[ticker] = tx_and_prices.round(3)
		
# check an individual stock
# MEGA_DICT['RUN'].tail()

# saving it, so we can access it quicker later
mega_dataset = pd.concat(MEGA_DICT.values(), axis=1)
# MEGA_DF.to_csv('../outputs/mega_df/MEGA_DF_{}.csv'.format(get_data(final_tickers, start_x, end_x)))  # optional

# like this:
# last_file = glob('../outputs/mega/MEGA*.csv')[-1] # path to file in the folder
# print(last_file[-(len(last_file))+(last_file.rfind('/')+1):])
# mega_dataset = pd.read_csv(last_file)
# mega_dataset['date'] = pd.to_datetime(mega_dataset['date'])
# mega_dataset.set_index('Date', inplace=True)

In [171]:
mega_dataset.columns
mega_dataset

,BTC-USD_close,BTC-USD_val_transact,BTC-USD_cml_units,BTC-USD_cml_cost,BTC-USD_gain_loss,BTC-USD_avg_price,BTC-USD_mktvalue,ETH-USD_close,ETH-USD_val_transact,ETH-USD_cml_units,...,AVAX-USD_gain_loss,AVAX-USD_avg_price,AVAX-USD_mktvalue,LTC-USD_close,LTC-USD_val_transact,LTC-USD_cml_units,LTC-USD_cml_cost,LTC-USD_gain_loss,LTC-USD_avg_price,LTC-USD_mktvalue
2022-01-01,0.000,50000.0,1.299,50000.0,47.500,38483.125,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000,10000.0,456.256,50000.0,47.500,109.588,0.000
2022-02-01,0.000,100000.0,2.457,100000.0,6167.208,40702.368,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000,20000.0,896.883,100000.0,1820.910,111.497,0.000
2022-03-01,0.000,150000.0,3.555,150000.0,11929.627,42196.143,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000,30000.0,1301.034,150000.0,11006.276,115.293,0.000
2022-04-01,0.000,200000.0,4.881,200000.0,-15882.633,40978.869,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000,40000.0,1820.969,200000.0,-24837.253,109.832,0.000
2022-05-01,0.000,250000.0,6.453,250000.0,-44788.089,38740.041,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000,50000.0,2551.841,250000.0,-75377.512,97.968,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-10,70587.883,1400000.0,48.753,1400000.0,1678819.728,28715.895,3441405.425,3543.737,0.0,0.0,...,0.0,NaN,0.0,96.700,280000.0,18347.982,1400000.0,12118.524,76.303,1774245.448
2024-04-11,70060.609,1400000.0,48.753,1400000.0,1678819.728,28715.895,3415699.007,3505.248,0.0,0.0,...,0.0,NaN,0.0,98.689,280000.0,18347.982,1400000.0,12118.524,76.303,1810745.854
2024-04-12,67195.867,1400000.0,48.753,1400000.0,1678819.728,28715.895,3276032.836,3243.035,0.0,0.0,...,0.0,NaN,0.0,86.270,280000.0,18347.982,1400000.0,12118.524,76.303,1582872.492
2024-04-13,63821.473,1400000.0,48.753,1400000.0,1678819.728,28715.895,3111519.336,3004.900,0.0,0.0,...,0.0,NaN,0.0,77.607,280000.0,18347.982,1400000.0,12118.524,76.303,1423924.168


Columns KPI and other relevant values

In [172]:
portf_allvalues = mega_dataset.filter(regex='mktvalue').fillna(0) #  getting just the market value of each ticker
portf_allvalues['portf_value'] = portf_allvalues.sum(axis=1) # summing all market values

# For the S&P500 price return
sp500 = pdr.get_data_yahoo('^GSPC', start_x, end_x)
# clean_header(sp500)

#getting the pct change
portf_allvalues = portf_allvalues.join(sp500['Close'], how='inner')
portf_allvalues.rename(columns={'Close': 'sp500_mktvalue'}, inplace=True)
portf_allvalues['ptf_value_pctch'] = (portf_allvalues['portf_value'].pct_change()*100).round(2)
portf_allvalues['sp500_pctch'] = (portf_allvalues['sp500_mktvalue'].pct_change()*100).round(2)
portf_allvalues['ptf_value_diff'] = (portf_allvalues['portf_value'].diff()).round(2)
portf_allvalues['sp500_diff'] = (portf_allvalues['sp500_mktvalue'].diff()).round(2)
# KPI's for portfolio
kpi_portfolio7d_abs = portf_allvalues.tail(7).ptf_value_diff.sum().round(2)
kpi_portfolio15d_abs = portf_allvalues.tail(15).ptf_value_diff.sum().round(2)
kpi_portfolio30d_abs = portf_allvalues.tail(30).ptf_value_diff.sum().round(2)
kpi_portfolio200d_abs = portf_allvalues.tail(200).ptf_value_diff.sum().round(2)
kpi_portfolio7d_pct = (kpi_portfolio7d_abs/portf_allvalues.tail(7).portf_value[0]).round(3)*100
kpi_portfolio15d_pct = (kpi_portfolio15d_abs/portf_allvalues.tail(15).portf_value[0]).round(3)*100
kpi_portfolio30d_pct = (kpi_portfolio30d_abs/portf_allvalues.tail(30).portf_value[0]).round(3)*100
kpi_portfolio200d_pct = (kpi_portfolio200d_abs/portf_allvalues.tail(200).portf_value[0]).round(3)*100
# KPI's for S&P500
kpi_sp500_7d_abs = portf_allvalues.tail(7).sp500_diff.sum().round(2)
kpi_sp500_15d_abs = portf_allvalues.tail(15).sp500_diff.sum().round(2)
kpi_sp500_30d_abs = portf_allvalues.tail(30).sp500_diff.sum().round(2)
kpi_sp500_200d_abs = portf_allvalues.tail(200).sp500_diff.sum().round(2)
kpi_sp500_7d_pct = (kpi_sp500_7d_abs/portf_allvalues.tail(7).sp500_mktvalue[0]).round(3)*100
kpi_sp500_15d_pct = (kpi_sp500_15d_abs/portf_allvalues.tail(15).sp500_mktvalue[0]).round(3)*100
kpi_sp500_30d_pct = (kpi_sp500_30d_abs/portf_allvalues.tail(30).sp500_mktvalue[0]).round(3)*100
kpi_sp500_200d_pct = (kpi_sp500_200d_abs/portf_allvalues.tail(200).sp500_mktvalue[0]).round(3)*100


[*********************100%***********************]  1 of 1 completed


Plotly Charts.

In [173]:

initial_date = '2022-01-01'  # do not use anything earlier than your first trade
plotlydf_portfval = portf_allvalues[portf_allvalues.index > initial_date]
plotlydf_portfval = plotlydf_portfval[['portf_value', 'sp500_mktvalue', 'ptf_value_pctch',
                                     'sp500_pctch', 'ptf_value_diff', 'sp500_diff']].reset_index().round(2)
# calculating cumulative growth since initial date
plotlydf_portfval['ptf_growth'] = plotlydf_portfval.portf_value/plotlydf_portfval['portf_value'].iloc[0]
plotlydf_portfval['sp500_growth'] = plotlydf_portfval.sp500_mktvalue/plotlydf_portfval['sp500_mktvalue'].iloc[0]
plotlydf_portfval.rename(columns={'index': 'date'}, inplace=True)  # needed for later

# Plotly part
CHART_THEME = 'plotly_white'  # others examples: seaborn, ggplot2, plotly_dark
chart_ptfvalue = go.Figure()  # generating a figure that will be updated in the following lines
chart_ptfvalue.add_trace(go.Scatter(x=plotlydf_portfval.date, y=plotlydf_portfval.portf_value,
                    mode='lines',  # you can also use "lines+markers", or just "markers"
                    name='Global Value'))
chart_ptfvalue.layout.template = CHART_THEME
chart_ptfvalue.layout.height=500
chart_ptfvalue.update_layout(margin = dict(t=50, b=50, l=25, r=25))  # this will help you optimize the chart space
chart_ptfvalue.update_layout(
#     title='Global Portfolio Value (USD $)',
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='Value: $ USD',
        titlefont_size=14,
        tickfont_size=12,
        ))
chart_ptfvalue.update_xaxes(rangeslider_visible=False)
chart_ptfvalue.update_layout(showlegend=False)
chart_ptfvalue.show()

Portfolio Value Visualized.

In [191]:
df = plotlydf_portfval[['date', 'ptf_growth', 'sp500_growth']].copy().round(3)
df['month'] = df.date.dt.month_name()  # date column should be formatted as datetime
df['weekday'] = df.date.dt.day_name()  # could be interesting to analyze weekday returns later
df['year'] = df.date.dt.year
df['weeknumber'] = df.date.dt.week    # could be interesting to try instead of timeperiod
df['timeperiod'] = df.year.astype(str) + ' - ' + df.date.dt.month.astype(str).str.zfill(2)

# getting the percentage change for each period. the first period will be NaN
sp = df.reset_index().groupby('timeperiod').last()['sp500_growth'].pct_change()*100
ptf = df.reset_index().groupby('timeperiod').last()['ptf_growth'].pct_change()*100
plotlydf_growth_compare = pd.merge(ptf, sp, on='timeperiod').reset_index().round(3)
plotlydf_growth_compare.head()

# Plotly part
fig_growth2 = go.Figure()
fig_growth2.layout.template = CHART_THEME
fig_growth2.add_trace(go.Bar(
    x=plotlydf_growth_compare.timeperiod,
    y=plotlydf_growth_compare.ptf_growth.round(2),
    name='Portfolio'
))
fig_growth2.add_trace(go.Bar(
    x=plotlydf_growth_compare.timeperiod,
    y=plotlydf_growth_compare.sp500_growth.round(2),
    name='S&P 500',
))
fig_growth2.update_layout(barmode='group')
fig_growth2.layout.height=300
fig_growth2.update_layout(margin = dict(t=50, b=50, l=25, r=25))
fig_growth2.update_layout(
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='% change',
        titlefont_size=13,
        tickfont_size=12,
        ))

fig_growth2.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99))
fig_growth2.show()

C:\Users\hp\AppData\Local\Temp\ipykernel_13024\2343947586.py:5: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.



KPI Visual Cards.

In [192]:
indicators_ptf = go.Figure()
indicators_ptf.layout.template = CHART_THEME

indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_portfolio7d_pct,
    number = {'suffix': " %"},
    title = {"text": "<br><span style='font-size:0.7em;color:gray'>7 Days</span>"},
    delta = {'position': "bottom", 'reference': kpi_sp500_7d_pct, 'relative': False},
    domain = {'row': 0, 'column': 0}))

indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_portfolio15d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>15 Days</span>"},
    delta = {'position': "bottom", 'reference': kpi_sp500_15d_pct, 'relative': False},
    domain = {'row': 1, 'column': 0}))

indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_portfolio30d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>30 Days</span>"},
    delta = {'position': "bottom", 'reference': kpi_sp500_30d_pct, 'relative': False},
    domain = {'row': 2, 'column': 0}))

indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_portfolio200d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>200 Days</span>"},
    delta = {'position': "bottom", 'reference': kpi_sp500_200d_pct, 'relative': False},
    domain = {'row': 3, 'column': 1}))

indicators_ptf.update_layout(
    grid = {'rows': 4, 'columns': 1, 'pattern': "independent"},
    margin=dict(l=50, r=50, t=30, b=30)
)

indicators_sp500 = go.Figure()
indicators_sp500.layout.template = CHART_THEME
indicators_sp500.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_sp500_7d_pct,
    number = {'suffix': " %"},
    title = {"text": "<br><span style='font-size:0.7em;color:gray'>7 Days</span>"},
    domain = {'row': 0, 'column': 0}))

indicators_sp500.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_sp500_15d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>15 Days</span>"},
    domain = {'row': 1, 'column': 0}))

indicators_sp500.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_sp500_30d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>30 Days</span>"},
    domain = {'row': 2, 'column': 0}))

indicators_sp500.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_sp500_200d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>200 Days</span>"},
    domain = {'row': 3, 'column': 1}))

indicators_sp500.update_layout(
    grid = {'rows': 4, 'columns': 1, 'pattern': "independent"},
    margin=dict(l=50, r=50, t=30, b=30)
)

In [193]:
all_ops

,Date,close,ticker,type,val_transact,quantity,prev_units,cml_units,prev_cost,cml_cost,...,prev_cost.1,cml_cost.1,cost_transact.1,cml_invested.1,cost_unit.1,cum_position_val.1,gain_loss.1,yield.1,avg_price.1,current_value.1
0,2022-01-01,38483.125000,BTC-USD,Buy,50000,1.299271,NaN,1.299271,NaN,50000,...,NaN,50000,47.5,49952.5,2688.278809,5.000000e+04,4.750000e+01,-0.999049,2688.278809,5.000000e+04
1,2022-02-01,43193.234375,BTC-USD,Buy,50000,1.157589,1.299271,2.456859,50000.0,100000,...,50000.0,100000,47.5,99952.5,5838.402344,1.042950e+05,4.342485e+03,-0.956555,2803.739990,1.042950e+05
2,2022-03-01,45538.675781,BTC-USD,Buy,50000,1.097968,1.157589,3.554827,50000.0,150000,...,50000.0,150000,47.5,149952.5,9844.928467,1.672440e+05,1.729152e+04,-0.884687,2963.040934,1.672440e+05
3,2022-04-01,37714.875000,BTC-USD,Buy,50000,1.325737,1.097968,4.880564,50000.0,200000,...,50000.0,200000,47.5,199952.5,10920.747070,1.891399e+05,-1.081261e+04,-1.054076,2904.827393,1.891399e+05
4,2022-05-01,31792.310547,BTC-USD,Buy,50000,1.572707,1.325737,6.453271,50000.0,250000,...,50000.0,250000,47.5,249952.5,9711.640015,1.845592e+05,-6.539332e+04,-1.261623,2712.327515,1.845592e+05
5,2022-06-01,19784.726562,BTC-USD,Buy,50000,2.527202,1.572707,8.980473,50000.0,300000,...,50000.0,300000,47.5,299952.5,6403.792969,1.514143e+05,-1.485382e+05,-1.495206,2438.156067,1.514143e+05
6,2022-07-01,23336.896484,BTC-USD,Buy,50000,2.142530,2.527202,11.123003,50000.0,350000,...,50000.0,350000,47.5,349952.5,11770.621338,2.885515e+05,-6.140100e+04,-1.175455,2330.064819,2.885515e+05
7,2022-08-01,20049.763672,BTC-USD,Buy,50000,2.493795,2.142530,13.616798,50000.0,400000,...,50000.0,400000,47.5,399952.5,12429.479492,3.166152e+05,-8.333727e+04,-1.208368,2233.017334,3.166152e+05
8,2022-09-01,19431.789062,BTC-USD,Buy,50000,2.573103,2.493795,16.189902,50000.0,450000,...,50000.0,450000,47.5,449952.5,11951.807739,3.206200e+05,-1.293325e+05,-1.287436,2132.457479,3.206200e+05
9,2022-10-01,20495.773438,BTC-USD,Buy,50000,2.439527,2.573103,18.629429,50000.0,500000,...,50000.0,500000,47.5,499952.5,15727.144775,4.297077e+05,-7.024477e+04,-1.140503,2076.483179,4.297077e+05


In [194]:
# getting the accumulated positions for our tickers
last_positions = all_ops.groupby(['ticker']).agg({'cml_units': 'last', 'cml_cost': 'last',
                                                'gain_loss': 'sum', 'val_transact': 'sum'}).reset_index()
curr_prices = []

for tick in last_positions['ticker']:
    # stonk = yf.get_data(tick)
    asset = (yf.download(tick, start = today))

    price = asset['Close']
    curr_prices.append(price)
    print(f'Done for {tick}')
		
last_positions['price'] = curr_prices  # adding it to our dataframe
# last_positions['current_value'] = (last_positions.price * last_positions.cml_units).round(2)  # and now we can calculate
last_positions['current_value'] = all_ops['current_value']
# last_positions['avg_price'] = (last_positions.cml_cost / last_positions.cml_units).round(2)  # and now we can calculate
last_positions['avg_price'] = all_ops['avg_price']
last_positions = last_positions.sort_values(by='current_value', ascending=False)  # sorting by current value

# Plotly part
donut_top = go.Figure()
donut_top.layout.template = CHART_THEME
donut_top.add_trace(go.Pie(labels=last_positions.head(15).ticker, values=last_positions.head(15).current_value))
donut_top.update_traces(hole=.4, hoverinfo="label+value+percent")
donut_top.update_traces(textposition='outside', textinfo='label+value')
donut_top.update_layout(showlegend=False)
donut_top.update_layout(margin = dict(t=50, b=50, l=25, r=25))
donut_top.show()

[*********************100%***********************]  1 of 1 completed
Done for BTC-USD
[*********************100%***********************]  1 of 1 completed
Done for LTC-USD


In [195]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.FLATLY])

app.layout = dbc.Container(
    [
        dbc.Row(dbc.Col(html.H2('Base 2 Portfolio Overview', className='text-center text-primary, mb-3'))),  # header row
        
        dbc.Row([  # start of second row
            dbc.Col([  # first column on second row
            html.H5('Total Portfolio Value ($USD)', className='text-center'),
            dcc.Graph(id='chrt-portfolio-main',
                      figure=chart_ptfvalue,
                      style={'height':550}),
            html.Hr(),
            ], width={'size': 8, 'offset': 0, 'order': 1}),  # width first column on second row
            dbc.Col([  # second column on second row
            html.H5('Portfolio', className='text-center'),
            dcc.Graph(id='indicators-ptf',
                      figure=indicators_ptf,
                      style={'height':550}),
            html.Hr()
            ], width={'size': 2, 'offset': 0, 'order': 2}),  # width second column on second row
            dbc.Col([  # third column on second row
            html.H5('S&P500', className='text-center'),
            dcc.Graph(id='indicators-sp',
                      figure=indicators_sp500,
                      style={'height':550}),
            html.Hr()
            ], width={'size': 2, 'offset': 0, 'order': 3}),  # width third column on second row
        ]),  # end of second row
        
        dbc.Row([  # start of third row
            dbc.Col([  # first column on third row
                html.H5('Monthly Return (%)', className='text-center'),
                dcc.Graph(id='chrt-portfolio-secondary',
                      figure=fig_growth2,
                      style={'height':380}),
            ], width={'size': 8, 'offset': 0, 'order': 1}),  # width first column on second row
            dbc.Col([  # second column on third row
                html.H5('Top 15 Holdings', className='text-center'),
                dcc.Graph(id='pie-top15',
                      figure = donut_top,
                      style={'height':380}),
            ], width={'size': 4, 'offset': 0, 'order': 2}),  # width second column on second row
        ])  # end of third row
        
    ], fluid=True)


if __name__ == "__main__":
    app.run_server(debug=True, port=8058)

c:\Users\hp\AppData\Local\Programs\Python\Python38\lib\site-packages\dash\dash.py:516: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8058/
